In [33]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Read data
data_path = "nurse_schedule_project2_data_large_VA.xlsx"
locations = pd.read_excel(data_path, sheet_name="locations", index_col = 0, header = None)
nurses = pd.read_excel(data_path, sheet_name="nurses", index_col = 0)
task_time = pd.read_excel(data_path, sheet_name="task_execution_time", index_col = 0).T

patients = pd.read_excel(data_path, sheet_name="patients", index_col = 0)
# Break comma separated strings into lists
for col in ['M', 'T', 'W','Th','F','S','Su']:
    patients[col] = patients[col].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
for col in ['skillset']:
    nurses[col] = nurses[col].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Transpose the dataframe to access patient information easily
patients = patients.T
nurses = nurses.T


# Sets and Parameters
N = nurses.columns.tolist()            # set of nurses
NursesSkills = nurses.loc['skillset']  # set of skills
P = patients.columns.tolist()          # set of patients
L = list(locations.index)         # set of locations
D = ['M', 'T', 'W', 'Th', 'F', 'S', 'Su']  # set of days
T_tasks = task_time.columns.tolist() # set of tasks

# Parameters


# define a function to list all patients in a given location
def find_keys_with_inner_value(df, target_value):
    return list(df.T[df.T['location'] == target_value].index)

print(NursesSkills)


id
Nurse_1                                          [wound care]
Nurse_2     [drawing blood, physical therapy, personal hyg...
Nurse_3                                    [physical therapy]
Nurse_4     [physical therapy, medication, drawing blood, ...
Nurse_5     [personal hygiene assistance, medication, phys...
Nurse_6     [physical therapy, administering injections, m...
Nurse_7     [medication, wound care, administering injecti...
Nurse_8     [physical therapy, personal hygiene assistance...
Nurse_9                [wound care, administering injections]
Nurse_10    [personal hygiene assistance, physical therapy...
Nurse_11                                         [wound care]
Nurse_12                                         [medication]
Nurse_13    [personal hygiene assistance, medication, phys...
Nurse_14    [medication, administering injections, wound c...
Nurse_15    [drawing blood, wound care, administering inje...
Nurse_16            [medication, personal hygiene assistance]
Nurse

In [34]:
# Assuming 'task_time' is a dictionary where keys are task names and values are durations
# and 'schedule' is a dictionary where keys are nurse names and values are dictionaries with days of the week as keys and list of patient IDs as values.

def calculate_daily_hours(schedule, task_time, patients):
    # Create a dictionary to hold total hours for each nurse each day
    nurse_daily_hours = {nurse: {day: 0 for day in D} for nurse in N}

    # For each nurse
    for nurse, days in schedule.items():
        # For each day
        for day, patient_list in days.items():
            # Calculate total hours for this nurse on this day
            for patient in patient_list:
                # Add hours for each task the patient needs on this day
                for task in patients[day][patient]:
                    nurse_daily_hours[nurse][day] += task_time[task]

    return nurse_daily_hours

# Now you'd run this function to get the hours for each nurse each day
# Then, you'd modify the schedule if any nurse exceeds 10 hours on any day

In [35]:
# Initialize a dictionary to hold the schedule
schedule = {nurse: {'M': [], 'T': [], 'W': [], 'Th': [], 'F': [], 'S': [], 'Su': []} for nurse in N}

# Assuming that 'task_time' is a DataFrame where index are task names and there's a 'Time' column.
# Convert task_time to a dictionary for easier use in the function if required.
task_time_dict = task_time.to_dict('index')

print(task_time_dict)

# Function to assign nurses to patients based on skillset matching
def assign_nurses_to_patients(nurses, patients, task_time_dict, max_hours_per_day=10):
    # Initialize a schedule dictionary
    schedule = {nurse: {day: [] for day in D} for nurse in nurses.columns}

    # Initialize a dictionary to keep track of the total hours worked by each nurse each day
    nurse_hours = {nurse: {day: 0 for day in D} for nurse in nurses.columns}

    # Iterate over the days and patients to assign tasks to nurses based on their skills
    for day in D:  # 'D' is the list of days
        for patient in P:  # 'P' is the list of patient IDs
            # Check if the patient's info for the day is a list indicating they have tasks that day
            if isinstance(patients.loc[day, patient], list):
                patient_tasks = patients.loc[day, patient]
                for task in patient_tasks:
                    for nurse in nurses.columns:
                        nurse_skills = nurses.loc['skillset', nurse]
                        # Access the duration for the task
                        task_duration = task_time_dict['Time'].get(task, 0) / 60  # Convert minutes to hours
                        if task in nurse_skills and (nurse_hours[nurse][day] + task_duration) <= max_hours_per_day:
                            # Assign task to nurse for this day
                            schedule[nurse][day].append((patient, task))
                            # Update hours for the nurse
                            nurse_hours[nurse][day] += task_duration

    return schedule, nurse_hours

# Use the function with the provided structure of task_time_dict
updated_schedule, nurse_hours = assign_nurses_to_patients(nurses, patients, task_time_dict)


print(updated_schedule)

{'Time': {'medication': 20, 'drawing blood': 89, 'physical therapy': 87, 'wound care': 50, 'administering injections': 40, 'personal hygiene assistance': 72}}
{'Nurse_1': {'M': [('Patient_7', 'wound care'), ('Patient_12', 'wound care'), ('Patient_14', 'wound care'), ('Patient_19', 'wound care'), ('Patient_20', 'wound care'), ('Patient_23', 'wound care'), ('Patient_25', 'wound care'), ('Patient_36', 'wound care'), ('Patient_41', 'wound care'), ('Patient_48', 'wound care'), ('Patient_54', 'wound care'), ('Patient_62', 'wound care')], 'T': [('Patient_7', 'wound care'), ('Patient_12', 'wound care'), ('Patient_14', 'wound care'), ('Patient_19', 'wound care'), ('Patient_20', 'wound care'), ('Patient_23', 'wound care'), ('Patient_25', 'wound care'), ('Patient_36', 'wound care'), ('Patient_41', 'wound care'), ('Patient_48', 'wound care'), ('Patient_54', 'wound care'), ('Patient_62', 'wound care')], 'W': [('Patient_8', 'wound care'), ('Patient_24', 'wound care'), ('Patient_27', 'wound care'), (

# TESTS

In [36]:
import unittest

class TestNurseScheduling(unittest.TestCase):
    def setUp(self):
        # Setup a dummy schedule and task times
        self.schedule = schedule  # Use the actual schedule variable after assignments
        self.task_time = task_time  # Use the actual task time dictionary

    def test_maximum_working_hours(self):
        # Calculate daily hours for nurses
        nurse_daily_hours = calculate_daily_hours(self.schedule, self.task_time, patients)
        # Test that no nurse works more than 10 hours a day
        for nurse, days in nurse_daily_hours.items():
            for day, hours in days.items():
                self.assertLessEqual(hours, 10, f"Nurse {nurse} is scheduled to work {hours} hours on {day}, exceeding the 10-hour limit.")

# Add more test cases if needed

if __name__ == '__main__':
    unittest.main()

usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument 'c:\\Users\\mcfro\\AppData\\Roaming\\jupyter\\runtime\\kernel-v2-17352opI2UcDa3O1u.json'


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
# City with needs on each day,
# array of 7 lists, each list contains patients in that city on that day 

# total task time / city / day 

# function to determine total task time / city / day
def calculate_total_hours_per_city_per_day(day, city): 
    total_hours = 0
    for patient in find_keys_with_inner_value(patients, city):
        for task in patients[patient][day]:
            total_hours += task_time[task]
    return total_hours

# function to determine total task time / city / day / task
def calculate_total_hours_per_city_per_day_per_task(day, city, task): 
    total_hours = 0
    for patient in find_keys_with_inner_value(patients, city):
        for theTask in patients[patient][day]:
            if theTask == task:
                total_hours += task_time[task]
    return total_hours


# make an array for each nurse with the total hours they work on each day (init to 0)
eachNursesHours = {}
for nurse in N:
    eachNursesHours[nurse] = {}
    for day in D:
        eachNursesHours[nurse][day] = 0

print(eachNursesHours)

# make an array for each city with the total hours of work needed on each day using the function above
eachCityHours = {}
for city in L:
    eachCityHours[city] = {}
    for day in D:
        eachCityHours[city][day] = {}
        for task in T_tasks:
            eachCityHours[city][day][task] = calculate_total_hours_per_city_per_day_per_task(day, city, task)

print()
print()
print()
print(eachCityHours)

# loop through each city for each day and assign a nurse to each task's total time if the nurse has the skillset and is available
# THIS FUNCTRION DOES NOT WORK YET @JOSH we will fix it later 
def assignNurses():
    for city in L:
        for day in D:
            for task in T_tasks:
                for nurse in N:
                    if task in NursesSkills['Nurse_1'] and eachNursesHours[nurse][day] < eachCityHours[city][day][task]:
                        eachNursesHours[nurse][day] += eachCityHours[city][day][task]
                        print("Nurse " + nurse + " assigned to " + task + " in " + city + " on " + day)
                        break

#assignNurses()


{'Nurse_1': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_2': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_3': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_4': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_5': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_6': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_7': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_8': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_9': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_10': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_11': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_12': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_13': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_14': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 